## Evaluate against Chat API using prepared data

### Update path and filenames

In [4]:
import requests
import json
import pandas as pd
from dotenv import load_dotenv

load_dotenv()


path_to_data = '../services/risk-data/'
input_dataset_name = 'prompt_injection.csv' # Update this value if you want to use a different dataset
output_dataset_name = 'prompt_injection_with_response.csv' # Update this value if you want to use a different name for the output dataset

pd.set_option('display.max_colwidth', None)
url = 'http://localhost:3000/chat'

### Load the dataset and pass each row to the Chat API

In [ ]:
df = pd.read_csv(f"{path_to_data}/{input_dataset_name}", sep=';', encoding='utf-8')

for index, row in df.iterrows():
    try:
        # Assuming chat_history is a stringified JSON, parse it
        chat_history_json = json.loads(row['chat_history'])
    except json.JSONDecodeError as e:
        print(f"Error parsing chat_history at row {index}: {e}")
        continue
    
    json_body = {
        "user_request": row['user_input'],
        "chat_history": chat_history_json,
        "user_id": 0
    }

    response = requests.post(url, headers={'accept': 'application/json', 'Content-type' : 'application/json'}, json=json_body)

    if response.status_code == 200:
        data = response.json()
        print("Data retrieved from the API:")
        print(data)
        df.at[index, 'agent_response'] = data
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        print("Response content:", response.content)


### Write dataframe to .csv file

In [3]:
df.to_csv(f"{path_to_data}/{output_dataset_name}", sep=';', encoding='utf-8', index=False)